In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from src.data_generator import *
from src.train import *
from src.utils import *
from src.networks import *

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]=str(0)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [4]:
model_save_dir='/home/garg/data/WeatherBench/predictions_P/saved_models/'
datadir='/home/garg/data/WeatherBench/5.625deg/'
pred_save_dir='/home/garg/data/WeatherBench/predictions_P/'
config_dir='/home/garg/WeatherBench/nn_configs/'

# model_save_dir='/home/rasp/data/myWeatherBench/predictions_P/saved_models/'
# datadir='/data/stephan/WeatherBench/5.625deg/'
# pred_save_dir='/home/rasp/data/myWeatherBench/predictions_P/'

# MC Dropout

In [20]:
# #ztt
# exp_id_path=f'{config_dir}P/024-resnet_d3_ztt_5d_dr0.0.yml'
# !python create_dropout_ensemble.py 3 {exp_id_path} {datadir} {model_save_dir} {pred_save_dir}

In [21]:
# # precipitation
# exp_id_path=f'{config_dir}P/033-resnet_d3_tp_5d_dr0.2.yml'
# !python create_dropout_ensemble.py 3 {exp_id_path} {datadir} {model_save_dir} {pred_save_dir}

# Parametric

In [5]:
args = load_args(f'{config_dir}P/027-resnet_d3_ztt_5d_param.yml')
args = load_args(f'{config_dir}P/005-resnet_d3_ztt_3d_param.yml')

In [6]:
args['model_save_dir']=model_save_dir
args['pred_save_dir']=pred_save_dir
args['datadir']=datadir

args['ext_mean'] = xr.open_dataarray(f'{args["model_save_dir"]}{args["exp_id"]}_mean.nc')
args['ext_std'] = xr.open_dataarray(f'{args["model_save_dir"]}{args["exp_id"]}_std.nc')

In [7]:
dg_test = load_data(**args, only_test=True)

In [8]:
def create_lat_crps(lat, n_vars, relu=False):
    weights_lat = np.cos(np.deg2rad(lat)).values
    weights_lat /= weights_lat.mean()
    def crps_loss(y_true, y_pred):
        # Split input
        mu = y_pred[:, :, :, :n_vars]
        sigma = y_pred[:, :, :, n_vars:]

        # To stop sigma from becoming negative we first have to
        # convert it the the variance and then take the square
        # root again.
        if relu:
            sigma = tf.nn.relu(sigma)
        else:
            sigma = tf.math.sqrt(tf.math.square(sigma))

        # The following three variables are just for convenience
        loc = (y_true - mu) / tf.maximum(1e-7, sigma)
        phi = 1.0 / np.sqrt(2.0 * np.pi) * tf.math.exp(-tf.math.square(loc) / 2.0)
        Phi = 0.5 * (1.0 + tf.math.erf(loc / np.sqrt(2.0)))
        # First we will compute the crps for each input/target pair
        crps =  sigma * (loc * (2. * Phi - 1.) + 2 * phi - 1. / np.sqrt(np.pi))
        crps = crps * weights_lat[None, : , None, None]

        # Then we take the mean. The cost is now a scalar
        return tf.reduce_mean(crps)
    return crps_loss

In [9]:
loss1 = create_lat_crps(dg_test.data.lat, len(dg_test.output_idxs), 
                       relu=True)

In [10]:
model = keras.models.load_model(
    f'{args["model_save_dir"]}/{args["exp_id"]}.h5', 
    custom_objects={'PeriodicConv2D': PeriodicConv2D, 'crps_loss':loss1})

In [12]:
pred=create_predictions(model, dg_test, parametric=True)

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'

In [ ]:
model.predict()

In [16]:
    dg=dg_test
    parametric=True
    no_mean=False
    verbose=0
    level_names = dg.data.isel(level=dg.output_idxs).level_names
    level = dg.data.isel(level=dg.output_idxs).level
    if parametric:
        # pdb.set_trace()
        lvl = level_names.values
        mm, ss = [], []
        for l in lvl:
            m = l.split('_'); s = l.split('_')
            m[0] += '-mean'; s[0] += '-std'
            mm.append('_'.join(m)); ss.append('_'.join(s))
        lvl = mm + ss
        level_names = xr.concat([level_names]*2, dim='level')
        level_names[:] = lvl
        level = xr.concat([level]*2, dim='level')

    

    mean = dg.mean.isel(level=dg.output_idxs).values if not no_mean else 0
    std = dg.std.isel(level=dg.output_idxs).values

    if dg.is_categorical:
        # preds = model.predict(dg.tfr_dataset or dg, verbose=verbose)
        preds = []
        for X, y in tqdm(dg):
            preds.append(model.predict(X))
        preds = np.concatenate(preds)
        if dg.predict_difference:
            unnormalized_bins = dg.bins * std[:, None]
        else:
            unnormalized_bins = dg.bins * std[:, None] + mean[:, None] 
        
        level_names = dg.data.isel(level=dg.output_idxs).level_names
        level = dg.data.isel(level=dg.output_idxs).level
        preds = xr.DataArray(
            preds,
            dims=['time', 'lat', 'lon', 'level', 'bin'],
            coords={'time': dg.valid_time, 'lat': dg.data.lat, 'lon': dg.data.lon,
                    'level': level,
                    'level_names': level_names,
                    'bin': np.arange(dg.num_bins),
                    },
        )
    else:
        preds = model.predict(dg.tfr_dataset or dg, verbose=verbose)
        preds = xr.DataArray(
            preds[0] if multi_dt else preds,
            dims=['time', 'lat', 'lon', 'level'],
            coords={'time': dg.valid_time, 'lat': dg.data.lat, 'lon': dg.data.lon,
                    'level': level,
                    'level_names': level_names
                    },
        )
        # Unnormalize

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'